# 🦜🔗 LangChain Playwright test creator

In [ ]:
%pip install playwright > /dev/null
%pip install  lxml

!playwright install

In [ ]:
import re
from langchain.agents.agent_toolkits import PlayWrightBrowserToolkit
from langchain.tools.playwright.utils import (
    create_async_playwright_browser,  # A synchronous browser is available, though it isn't compatible with jupyter.
)
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.tools import StructuredTool
from langchain.schema.output_parser import StrOutputParser



# This import is required only for jupyter notebooks, since they have their own eventloop
import asyncio
import nest_asyncio

nest_asyncio.apply()

async_browser = create_async_playwright_browser()
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()
tools_by_name = {tool.name: tool for tool in tools}
get_elements_tool = tools_by_name["get_elements"]



In [ ]:

llm=AzureChatOpenAI(deployment_name="gpt-4-32k", 
                       api_version="2023-07-01-preview",
                       temperature=0)

output_parser = StrOutputParser()

tools_by_name = {tool.name: tool for tool in tools}
current_webpage_tool = tools_by_name["current_webpage"]

async def get_source(url):
    page = await current_webpage_tool.async_browser.new_page()
    await page.goto(url)
    content = await page.content()
    return content

gen_test_prompt = PromptTemplate(
    template="""
You are a QA assistant AI generating typescript playwright code to provide end to end tests
for a web application. 
Please create a playwright typescript test for the url {url}.
The test should provide this functionality: {functionality}
Return the typescript code using markup notation.
The source code for the url above is: 
------------------- BEGIN SOURCE CODE -------------------
{source}
------------------- END SOURCE CODE -------------------
""",
    input_variables=["url", "functionality", "source"])


def create_playwright_test(url: str, functionality: str) -> str:
    """ Writes a playwright test for a given url and a detailed description of the functionality of this test. """
    loop = asyncio.get_event_loop()
    source = loop.run_until_complete(get_source(url))
    chain = gen_test_prompt | llm | output_parser
    result = chain.run(url=url, functionality=functionality, source=source)
    code = re.search("```(.+?)\n(.+)```" , result, re.DOTALL).group(2)
    return code



create_test = StructuredTool.from_function(create_playwright_test)

tools.append(create_test)

In [ ]:
# Structured tools are compatible with the STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION agent type.
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

result = await agent_chain.arun("Have a look at the page https://bahn.de. Recommend 3 end-to-end tests for this page for playwright and create them.")
print(result)